# DCGAN

In [ ]:
import os, sys
import torch, torchvision
import torch.nn as nn

from torchvision import transforms
from torchvision.utils import save_image

sys.path.append(os.pardir)
from utils import *

print(torch.__version__)

### Settings

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

image_size = 28*28
hidden_size = 256
batch_size = 100
latent_size = 64

num_epochs = 20
learning_rate = 0.0002

In [ ]:
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5),
                        std=(0.5, 0.5, 0.5))
])

In [ ]:
FMNIST_dataset = torchvision.datasets.FashionMNIST(root='./data/FMNIST/',
                                                  train=True,
                                                  transform=transform,
                                                  download=True)

FMNIST_loader = torch.utils.data.DataLoader(dataset=FMNIST_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

In [ ]:
MNIST_dataset = torchvision.datasets.MNIST(root='./data/MNIST/',
                                                  train=True,
                                                  transform=transform,
                                                  download=True)

MNIST_loader = torch.utils.data.DataLoader(dataset=MNIST_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

In [ ]:
it = iter(MNIST_loader)
(images, labels) = it.next()

print(images.size())

In [ ]:
sample_dir = 'samples/DCGAN/MNIST/'

if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

### Functions

In [ ]:
def reset_grad():
    G_Optim.zero_grad()
    D_Optim.zero_grad()
    
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

### Model

In [ ]:
G = nn.Sequential(
    nn.ConvTranspose2d(latent_size, 1024, 4, 1, 0, bias=False),
    nn.BatchNorm2d(1024),
    nn.ReLU(),
    nn.ConvTranspose2d(1024, 512, 4, 2, 1, bias=False),
    nn.BatchNorm2d(512),
    nn.ReLU(),
    nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
    nn.BatchNorm2d(256),
    nn.ReLU(),
    nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.ConvTranspose2d(128, 1, 4, 2, 1, bias=False),
    nn.Tanh()
).to(device)

D = nn.Sequential(
    nn.Conv2d(1, 128, 4, 2, 1, bias=False),
    nn.LeakyReLU(0.2),
    nn.Conv2d(128, 256, 4, 2, 1, bias=False),
    nn.BatchNorm2d(256),
    nn.LeakyReLU(0.2),
    nn.Conv2d(256, 512, 4, 2, 1, bias=False),
    nn.BatchNorm2d(512),
    nn.LeakyReLU(0.2),
    nn.Conv2d(512, 1024, 4, 2, 1, bias=False),
    nn.BatchNorm2d(1024),
    nn.LeakyReLU(0.2),
    nn.Conv2d(1024, latent_size, 4, 1, 0, bias=False),
    nn.Sigmoid()
).to(device)

In [ ]:
D.apply(weights_init)
G.apply(weights_init)

### Loss Function & Optimizer

In [ ]:
criterion = nn.BCELoss()
G_Optim = torch.optim.Adam(G.parameters(), lr=learning_rate, betas=(0.5, 0.999))
D_Optim = torch.optim.Adam(D.parameters(), lr=learning_rate, betas=(0.5, 0.999))

### Train

In [ ]:
G_Losses = []
D_Losses = []

Dx = []
DGz = []

total_step = len(MNIST_loader)
for epoch in range(num_epochs):
    for i, ( images, _ ) in enumerate(MNIST_loader):
        images = images.to(device)
        
        real_labels = torch.ones(batch_size, latent_size, 1, 1).to(device)
        fake_labels = torch.zeros(batch_size, latent_size, 1, 1).to(device)
        
        #
        #  D
        #
        outputs = D(images)     
        D_Loss_Real = criterion(outputs, real_labels)
        real_score = outputs
        
        z = torch.randn(batch_size, latent_size, 1, 1).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        D_Loss_Fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        D_Loss = D_Loss_Real + D_Loss_Fake
        reset_grad()
        D_Loss.backward()
        D_Optim.step()
        
        #
        #  G
        #
        
        z = torch.randn(batch_size, latent_size, 1, 1).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        G_Loss = criterion(outputs, real_labels)
        
        reset_grad()
        G_Loss.backward()
        G_Optim.step()
        
        if (i+1) % 20 == 0:
             print('Epoch [{}/{}], Step [{}/{}], D_Loss : {:.4f}, G_Loss : {:.4f}, D(x) : {:.2f}, D(G(z)) : {:.2f}'
                 .format(epoch+1, num_epochs, i+1, total_step, D_Loss.item(), G_Loss.item(), real_score.mean().item(), fake_score.mean().item()))
       
    if (epoch + 1) == 1:
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
        
    D_Losses.append(D_Loss.item())
    G_Losses.append(G_Loss.item())
    
    Dx.append(real_score.mean().item())
    DGz.append(fake_score.mean().item())
        
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

### Result

In [ ]:
drawLoss({'G_Loss':G_Losses, 'D_Loss':D_Losses})

In [ ]:
drawLoss({'Dx':Dx, 'DGz':DGz})